### Ньяти Каелиле БВТ2201
### Курсовая Работа - СИАОД

### 1. Timetable for 12 hour night shift drivers only (2 weeks)

In [11]:
from collections import deque

def generate_two_week_timetable():
    # Initialize drivers
    drivers = deque(["Driver_1", "Driver_2", "Driver_3"])  # Circular queue
    schedule = []  # Store the timetable
    days = [
        "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)",
        "Thursday (Week 1)", "Friday (Week 1)", "Saturday (Week 1)", "Sunday (Week 1)",
        "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
        "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
    ]
    rest_pool = deque()  # Tracks drivers currently resting

    # Assign shifts for two weeks
    for day in days:
        # Rotate drivers back to active pool if they've rested for 2 days
        if rest_pool and len(rest_pool[0]["rest_days"]) == 2:
            rested_driver = rest_pool.popleft()["driver"]
            drivers.append(rested_driver)
        
        # Assign the next driver for the night shift
        current_driver = drivers.popleft()
        schedule.append({"day": day, "driver": current_driver, "shift": "18:00-6:00"})
        
        # Move the driver to the rest pool
        rest_pool.append({"driver": current_driver, "rest_days": []})
        
        # Increment rest days for drivers in the rest pool
        for resting_driver in rest_pool:
            resting_driver["rest_days"].append(1)

    return schedule

# Generate the timetable
timetable = generate_two_week_timetable()

# Print the timetable
for entry in timetable:
    print(f"{entry['day']}: {entry['driver']} - {entry['shift']}")


Monday (Week 1): Driver_1 - 18:00-6:00
Tuesday (Week 1): Driver_2 - 18:00-6:00
Wednesday (Week 1): Driver_3 - 18:00-6:00
Thursday (Week 1): Driver_1 - 18:00-6:00
Friday (Week 1): Driver_2 - 18:00-6:00
Saturday (Week 1): Driver_3 - 18:00-6:00
Sunday (Week 1): Driver_1 - 18:00-6:00
Monday (Week 2): Driver_2 - 18:00-6:00
Tuesday (Week 2): Driver_3 - 18:00-6:00
Wednesday (Week 2): Driver_1 - 18:00-6:00
Thursday (Week 2): Driver_2 - 18:00-6:00
Friday (Week 2): Driver_3 - 18:00-6:00
Saturday (Week 2): Driver_1 - 18:00-6:00
Sunday (Week 2): Driver_2 - 18:00-6:00


### 2. Timetable for 12 hour Day and Night shifts drivers (2 weeks)

In [12]:
from collections import deque

def generate_timetable_with_day_shift_and_night_shift():
    # Initialize drivers for night and day shifts
    night_drivers = deque(["Driver_1", "Driver_2", "Driver_3"])  # Night-shift drivers
    day_drivers = deque(["Day_Driver_1", "Day_Driver_2", "Day_Driver_3"])  # Day-shift drivers
    schedule = []  # Store the timetable
    days = [
        "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
        "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
        "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
    ]
    
    night_rest_pool = deque()  # Resting night-shift drivers
    day_rest_pool = deque()  # Resting day-shift drivers

    # Assign shifts for two weeks
    for day in days:
        # Night shift assignment
        if night_rest_pool and len(night_rest_pool[0]["rest_days"]) == 2:
            rested_driver = night_rest_pool.popleft()["driver"]
            night_drivers.append(rested_driver)

        current_night_driver = night_drivers.popleft()
        schedule.append({"day": day, "driver": current_night_driver, "shift": "18:00-6:00 (Night)"})
        night_rest_pool.append({"driver": current_night_driver, "rest_days": []})

        # Day shift assignment (on all days including weekdays and weekends)
        if "Saturday" in day or "Sunday" in day or ("Monday" in day or "Tuesday" in day or "Wednesday" in day or "Thursday" in day or "Friday" in day):
            # Rotate day drivers back to active pool if rested
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            # Ensure we have an available day driver
            if day_drivers:
                current_day_driver = day_drivers.popleft()
                schedule.append({"day": day, "driver": current_day_driver, "shift": "6:00-18:00 (Day)"})
                day_rest_pool.append({"driver": current_day_driver, "rest_days": []})

        # Increment rest days for night-shift drivers
        for resting_driver in night_rest_pool:
            resting_driver["rest_days"].append(1)

        # Increment rest days for day-shift drivers
        for resting_driver in day_rest_pool:
            resting_driver["rest_days"].append(1)

    return schedule

# Generate the updated timetable
timetable = generate_timetable_with_day_shift_and_night_shift()

# Print the combined timetable for both day and night shifts over two weeks
for entry in timetable:
    print(f"{entry['day']}: {entry['driver']} - {entry['shift']}")


Monday (Week 1): Driver_1 - 18:00-6:00 (Night)
Monday (Week 1): Day_Driver_1 - 6:00-18:00 (Day)
Tuesday (Week 1): Driver_2 - 18:00-6:00 (Night)
Tuesday (Week 1): Day_Driver_2 - 6:00-18:00 (Day)
Wednesday (Week 1): Driver_3 - 18:00-6:00 (Night)
Wednesday (Week 1): Day_Driver_3 - 6:00-18:00 (Day)
Thursday (Week 1): Driver_1 - 18:00-6:00 (Night)
Thursday (Week 1): Day_Driver_1 - 6:00-18:00 (Day)
Friday (Week 1): Driver_2 - 18:00-6:00 (Night)
Friday (Week 1): Day_Driver_2 - 6:00-18:00 (Day)
Saturday (Week 1): Driver_3 - 18:00-6:00 (Night)
Saturday (Week 1): Day_Driver_3 - 6:00-18:00 (Day)
Sunday (Week 1): Driver_1 - 18:00-6:00 (Night)
Sunday (Week 1): Day_Driver_1 - 6:00-18:00 (Day)
Monday (Week 2): Driver_2 - 18:00-6:00 (Night)
Monday (Week 2): Day_Driver_2 - 6:00-18:00 (Day)
Tuesday (Week 2): Driver_3 - 18:00-6:00 (Night)
Tuesday (Week 2): Day_Driver_3 - 6:00-18:00 (Day)
Wednesday (Week 2): Driver_1 - 18:00-6:00 (Night)
Wednesday (Week 2): Day_Driver_1 - 6:00-18:00 (Day)
Thursday (Week 2

### 3. Timetable for 8 hour drivers & 12 hour Day and Night shift drivers (2 weeks)

In [13]:
from collections import deque

def generate_timetable_with_all_shifts():
    # Initialize drivers for night and day shifts (12-hour)
    night_drivers = deque(["12hr_Driver_1", "12hr_Driver_2", "12hr_Driver_3"])  # 3 Night-shift drivers
    day_drivers = deque(["Day_Shift_Driver_1", "Day_Shift_Driver_2", "Day_Shift_Driver_3"])  # 3 Day-shift drivers
    eight_hour_drivers = [
        "8hr_Driver_1", "8hr_Driver_2",  # 6:00-14:00
        "8hr_Driver_3", "8hr_Driver_4"   # 10:00-18:00
    ]
    
    schedule = []  # Store the timetable
    days = [
        "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
        "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
        "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
    ]
    
    # Rest pools for night shift and day shift drivers
    night_rest_pool = deque()
    day_rest_pool = deque()

    for day in days:
        # Night shift assignment
        if night_rest_pool and len(night_rest_pool[0]["rest_days"]) == 2:
            rested_driver = night_rest_pool.popleft()["driver"]
            night_drivers.append(rested_driver)

        current_night_driver = night_drivers.popleft()
        schedule.append(f"{day}:")
        schedule.append(f"  {current_night_driver} - night shift (18:00 to 6:00)")

        night_rest_pool.append({"driver": current_night_driver, "rest_days": []})

        # 8-hour day shift assignment (6:00-18:00 and 10:00-18:00)
        if "Saturday" not in day and "Sunday" not in day:  # Only Monday to Friday
            for i in range(4):  # First four 8-hour drivers (6:00-18:00 and 10:00-18:00)
                if i < 2:
                    current_8hr_driver = eight_hour_drivers[i]
                    schedule.append(f"  {current_8hr_driver} - day shift (6:00 to 14:00)")
                else:
                    current_8hr_driver = eight_hour_drivers[i]
                    schedule.append(f"  {current_8hr_driver} - day shift (10:00 to 18:00)")

        # 12-hour day shift assignment (for Monday-Friday)
        if "Saturday" not in day and "Sunday" not in day:
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            if day_drivers:
                current_day_shift_driver = day_drivers.popleft()
                schedule.append(f"  {current_day_shift_driver} - day shift (6:00 to 18:00)")

                day_rest_pool.append({"driver": current_day_shift_driver, "rest_days": []})

        # Day shift assignment for weekends (Saturdays and Sundays)
        if "Saturday" in day or "Sunday" in day:
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            if day_drivers:
                current_day_driver = day_drivers.popleft()
                schedule.append(f"  {current_day_driver} - day shift (6:00 to 18:00)")

                day_rest_pool.append({"driver": current_day_driver, "rest_days": []})

        # Increment rest days for night-shift drivers
        for resting_driver in night_rest_pool:
            resting_driver["rest_days"].append(1)

        # Increment rest days for day-shift drivers
        for resting_driver in day_rest_pool:
            resting_driver["rest_days"].append(1)

    return schedule


# Generate the updated timetable
timetable = generate_timetable_with_all_shifts()

# Print the combined timetable in the requested format
for entry in timetable:
    print(entry)


Monday (Week 1):
  12hr_Driver_1 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (6:00 to 14:00)
  8hr_Driver_2 - day shift (6:00 to 14:00)
  8hr_Driver_3 - day shift (10:00 to 18:00)
  8hr_Driver_4 - day shift (10:00 to 18:00)
  Day_Shift_Driver_1 - day shift (6:00 to 18:00)
Tuesday (Week 1):
  12hr_Driver_2 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (6:00 to 14:00)
  8hr_Driver_2 - day shift (6:00 to 14:00)
  8hr_Driver_3 - day shift (10:00 to 18:00)
  8hr_Driver_4 - day shift (10:00 to 18:00)
  Day_Shift_Driver_2 - day shift (6:00 to 18:00)
Wednesday (Week 1):
  12hr_Driver_3 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (6:00 to 14:00)
  8hr_Driver_2 - day shift (6:00 to 14:00)
  8hr_Driver_3 - day shift (10:00 to 18:00)
  8hr_Driver_4 - day shift (10:00 to 18:00)
  Day_Shift_Driver_3 - day shift (6:00 to 18:00)
Thursday (Week 1):
  12hr_Driver_1 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (6:00 to 14:00)
  8hr_Driver_2 - day shift (6:

### 4. Adding lunch and 15-mins breaks to the timetable

In [14]:
import random
from collections import deque
import numpy as np

def generate_timetable_with_all_shifts():
    # Initialize drivers for night and day shifts (12-hour)
    night_drivers = deque(["12hr_Driver_1", "12hr_Driver_2", "12hr_Driver_3"])  # 3 Night-shift drivers
    day_drivers = deque(["12hr_Day_Driver_1", "12hr_Day_Driver_2", "12hr_Day_Driver_3"])  # 3 Day-shift drivers
    eight_hour_drivers = [
        "8hr_Driver_1", "8hr_Driver_2",  # 6:00-14:00
        "8hr_Driver_3", "8hr_Driver_4"   # 10:00-18:00
    ]
    
    schedule = []  # Store the timetable
    days = [
        "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
        "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
        "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
    ]
    
    # Rest pools for night shift and day shift drivers
    night_rest_pool = deque()
    day_rest_pool = deque()

    # Store the working drivers for each day (now includes shift times)
    day_drivers_schedule = {}

    # Assigning drivers for weekday (Monday-Friday) and weekend (Saturday-Sunday) 12-hour shifts
    for day in days:
        schedule.append(f"{day}:")
        day_drivers_schedule[day] = []  # Initialize the list for each day's working drivers

        # Reset lunch and break assignments at the start of each day
        lunch_drivers = []  # Reset lunch drivers for the day

        # Night shift assignment (if applicable)
        if night_rest_pool and len(night_rest_pool[0]["rest_days"]) == 2:
            rested_driver = night_rest_pool.popleft()["driver"]
            night_drivers.append(rested_driver)

        current_night_driver = night_drivers.popleft()
        schedule.append(f"  {current_night_driver} - night shift (18:00 to 6:00)")
        night_rest_pool.append({"driver": current_night_driver, "rest_days": []})

        # 8-hour day shift assignment (6:00-18:00 and 10:00-18:00)
        if "Saturday" not in day and "Sunday" not in day:  # Only Monday to Friday
            morning_shift_start_time = 6 * 60  # Start at 06:00 in minutes
            late_shift_start_time = (9 * 60) + 45  # Start at 10:00 in minutes

            for i in range(4):  # First four 8-hour drivers (6:00-18:00 and 10:00-18:00)
                if i < 2:
                    current_8hr_driver = eight_hour_drivers[i]
                    start_hour = morning_shift_start_time // 60
                    start_minute = morning_shift_start_time % 60
                    schedule.append(f"  {current_8hr_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02})")
                    day_drivers_schedule[day].append((current_8hr_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02}"))
                    morning_shift_start_time += 15  # Increment by 15 minutes for the next driver
                else:
                    current_8hr_driver = eight_hour_drivers[i]
                    start_hour = late_shift_start_time // 60
                    start_minute = late_shift_start_time % 60
                    schedule.append(f"  {current_8hr_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02})")
                    day_drivers_schedule[day].append((current_8hr_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02}"))
                    late_shift_start_time += 15  # Increment by 15 minutes for the next driver

        # 12-hour day shift assignment (for Monday-Friday)
        if "Saturday" not in day and "Sunday" not in day:
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            if day_drivers:
                # First 12-hour shift driver (starts at 06:00)
                current_day_shift_driver = day_drivers.popleft()
                start_hour = morning_shift_start_time // 60
                start_minute = morning_shift_start_time % 60
                schedule.append(f"  {current_day_shift_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02})")
                day_drivers_schedule[day].append((current_day_shift_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02}"))
                start_hour = (start_hour + 12) % 24  # Move the start hour 12 hours ahead for the next shift

                day_rest_pool.append({"driver": current_day_shift_driver, "rest_days": []})


        # Day shift assignment for weekends (Saturdays and Sundays)
        if "Saturday" in day or "Sunday" in day:
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            if day_drivers:
                # First 12-hour shift driver (starts at 06:00)
                current_day_driver = day_drivers.popleft()
                start_hour = 6  # Start at 06:00
                start_minute = 0  # Start at 00 minutes
                schedule.append(f"  {current_day_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02})")
                day_drivers_schedule[day].append((current_day_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02}"))
                start_hour = (start_hour + 12) % 24  # Move the start hour 12 hours ahead for the next shift

                day_rest_pool.append({"driver": current_day_driver, "rest_days": []})




        # Increment rest days for night-shift drivers
        for resting_driver in night_rest_pool:
            resting_driver["rest_days"].append(1)

        # Increment rest days for day-shift drivers
        for resting_driver in day_rest_pool:
            resting_driver["rest_days"].append(1)

        # Assign staggered lunch breaks
        lunch_drivers = assign_lunch_breaks(schedule, day_drivers_schedule[day])

        # Assign staggered 15-minute breaks for drivers without lunch
        assign_staggered_15_minute_breaks(schedule, lunch_drivers, day_drivers_schedule[day])

          # Assign staggered 15-minute breaks for night-shift drivers
        assign_staggered_15_minute_breaks_night(
            schedule,
            [(current_night_driver, "18:00", "6:00")]
        )

    return schedule


def assign_lunch_breaks(schedule, working_drivers):
    """
    Assign lunch breaks based on the day's working drivers.
    """
    
    lunch_drivers_today = []
            
    twelve_hour_day_driver = [
    (driver, start_time) for driver, start_time in working_drivers if "12hr" in driver
]
    if twelve_hour_day_driver:
        for driver, start_time in twelve_hour_day_driver:
            # Extract the actual start time (before " to")
            actual_start_time = start_time.split(" to ")[0]
            
            if actual_start_time == "06:00":
                # Assign lunch break from 13:00 to 14:00 for drivers starting at 06:00
                schedule.append(f"  {driver} - lunch break (13:00 to 14:00)")
                lunch_drivers_today.append(driver)
            else:
                # Assign lunch break from 13:30 to 14:30 for other start times
                schedule.append(f"  {driver} - lunch break (13:30 to 14:30)")
                lunch_drivers_today.append(driver)


    """
    # Assign lunch for one of the 10:00–18:00 shift drivers (14:00 to 15:00)
    afternoon_drivers = [
        driver for driver, start_time in working_drivers if int(start_time.split(":")[0]) == 10
    ]
    
    if afternoon_drivers:
        # Assign lunch to the first afternoon shift driver from 14:00 to 15:00
        schedule.append(f"  {afternoon_drivers[0]} - lunch break (14:00 to 15:00)")
        lunch_drivers_today.append(afternoon_drivers[0])
    """
    return lunch_drivers_today


def assign_staggered_15_minute_breaks(schedule, lunch_drivers, working_drivers):
    """
    Assign staggered 15-minute breaks to drivers not assigned lunch breaks.
    """
    # Initialize a list to store break assignments
    break_assignments = []
    
    # We will use a dictionary to track assigned break times to avoid conflicts
    assigned_breaks = {}

    # Filter out drivers who have lunch breaks
    eligible_drivers = [
        driver for driver, _ in working_drivers if driver not in lunch_drivers
    ]
    
    for driver, start_time in working_drivers:
        if driver not in eligible_drivers:
            continue

        # Extract the shift start time (assumes "HH:MM to HH:MM" format)
        if " to " in start_time:
            start_time = start_time.split(" to ")[0]  # Keep only the start time

        try:
            # Parse shift start time
            driver_start_hour = int(start_time.split(":")[0])
            driver_start_minute = int(start_time.split(":")[1])
        except ValueError:
            print(f"Skipping invalid start time for driver {driver}: {start_time}")
            continue

      # Dynamically calculate the end hour based on the start time and 8-hour shift
        driver_end_hour = driver_start_hour + 8
        driver_end_minute = driver_start_minute  # Assuming no additional minutes (full 8 hours)

        # Adjust the end time if the hours exceed 24 (wrap around)
        if driver_end_hour >= 24:
            driver_end_hour -= 24  # Wrap around to next day

        # List to store the assigned breaks for this driver
        breaks = []

        # Assign the first break (1hr 45mins after the start of the shift)
        current_hour = driver_start_hour
        current_minute = driver_start_minute + 105
        if current_minute >= 60:
            current_hour += current_minute // 60
            current_minute %= 60

        # Add the first break
        breaks.append((current_hour, current_minute))

        # Now assign up to 4 more breaks, spaced 1hr 45mins after the previous break's end time
        while len(breaks) < 6:
            last_break_hour, last_break_minute = breaks[-1]
            current_hour = last_break_hour
            current_minute = last_break_minute + 15 + 105  # Add 15 mins (break duration) + 1hr 45mins
            if current_minute >= 60:
                current_hour += current_minute // 60
                current_minute %= 60

            # Ensure the break fits within the shift's end time
            if current_hour >= driver_end_hour and (current_hour > driver_end_hour or current_minute > driver_end_minute):
                break

            # Add the next break
            breaks.append((current_hour, current_minute))
                
        # Assign breaks to the schedule while ensuring no overlap
        for break_hour, break_minute in breaks:
            formatted_start = f"{break_hour % 24:02}:{break_minute:02}"
            formatted_end_minute = break_minute + 15
            formatted_end_hour = break_hour
            if formatted_end_minute >= 60:
                formatted_end_hour += formatted_end_minute // 60
                formatted_end_minute %= 60
            formatted_end = f"{formatted_end_hour % 24:02}:{formatted_end_minute:02}"

            # Ensure the break time doesn't overlap with another driver
            while formatted_start in assigned_breaks:
                # If there's a conflict, move the break 15 minutes later
                break_minute += 15
                if break_minute >= 60:
                    break_hour += 1
                    break_minute -= 60
                formatted_start = f"{break_hour % 24:02}:{break_minute:02}"
                formatted_end_minute = break_minute + 15
                formatted_end_hour = break_hour
                if formatted_end_minute >= 60:
                    formatted_end_hour += formatted_end_minute // 60
                    formatted_end_minute %= 60
                formatted_end = f"{formatted_end_hour % 24:02}:{formatted_end_minute:02}"

            # Add the break to the schedule and track the assigned time
            break_assignments.append(f"  {driver} - 15-minute break ({formatted_start} to {formatted_end})")
            assigned_breaks[formatted_start] = driver
    
    # Append the break assignments to the schedule
    schedule.extend(break_assignments)


def assign_staggered_15_minute_breaks_night(schedule, night_drivers):
    """
    Assign staggered 15-minute breaks to night-shift drivers.
    Ensure no two drivers are on break at the same time.
    """
    break_assignments = []  # Store break assignments
    assigned_breaks = {}  # Track assigned break times to avoid conflicts

    for driver, shift_start, shift_end in night_drivers:
        # Parse the shift start and end times
        start_hour = int(shift_start.split(":")[0])
        start_minute = int(shift_start.split(":")[1])
        end_hour = int(shift_end.split(":")[0]) + 24  # Convert to 24-hour range for overnight shifts
        end_minute = int(shift_end.split(":")[1])

        # Initialize the current time for breaks (1hr 45mins after shift start)
        current_hour = start_hour
        current_minute = start_minute + 105  # Add 1hr 45mins
        if current_minute >= 60:
            current_hour += current_minute // 60
            current_minute %= 60

        # List to store this driver's assigned breaks
        breaks = []

        # Assign breaks at 1.5-hour intervals until the end of the shift
        while current_hour + current_minute / 60 < end_hour + end_minute / 60:
            # Format the current break start and end times
            break_start = f"{current_hour % 24:02}:{current_minute:02}"
            break_end_minute = current_minute + 15  # Add 15 minutes for the break duration
            break_end_hour = current_hour
            if break_end_minute >= 60:
                break_end_hour += break_end_minute // 60
                break_end_minute %= 60
            break_end = f"{break_end_hour % 24:02}:{break_end_minute:02}"

            # Ensure no overlap with other drivers
            while break_start in assigned_breaks:
                # Move the break 15 minutes later if there's a conflict
                current_minute += 15
                if current_minute >= 60:
                    current_hour += current_minute // 60
                    current_minute %= 60
                break_start = f"{current_hour % 24:02}:{current_minute:02}"
                break_end_minute = current_minute + 15
                break_end_hour = current_hour
                if break_end_minute >= 60:
                    break_end_hour += break_end_minute // 60
                    break_end_minute %= 60
                break_end = f"{break_end_hour % 24:02}:{break_end_minute:02}"

            # Assign the break
            breaks.append(f"  {driver} - 15-minute break ({break_start} to {break_end})")
            assigned_breaks[break_start] = driver

            # Increment time by 1hr 45mins for the next break (from the end of the current break)
            current_minute += 15 + 105  # Add 15 minutes (current break duration) + 1hr 45mins
            if current_minute >= 60:
                current_hour += current_minute // 60
                current_minute %= 60

        # Add this driver's breaks to the overall assignments
        break_assignments.extend(breaks)

    # Append the break assignments to the schedule
    schedule.extend(break_assignments)




def calculate_route_duration(time_of_day):
    """
    Calculate the route duration based on the time of day.
    Rush hours add 5 minutes; normal variability is ±10 minutes.
    """
    base_duration = 90  # 1.5 hours in minutes
    variability = random.randint(-10, 10)
    rush_hour_extra = 5 if time_of_day in ["7:00", "8:00", "09:00", "17:00", "18:00", "19:00"] else 0
    return base_duration + variability + rush_hour_extra


# Generate the updated timetable
timetable = generate_timetable_with_all_shifts()

# Print the combined timetable in the requested format
for entry in timetable:
    print(entry)


Monday (Week 1):


  12hr_Driver_1 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (06:00 to 14:00)
  8hr_Driver_2 - day shift (06:15 to 14:15)
  8hr_Driver_3 - day shift (09:45 to 17:45)
  8hr_Driver_4 - day shift (10:00 to 18:00)
  12hr_Day_Driver_1 - day shift (06:30 to 18:30)
  12hr_Day_Driver_1 - lunch break (13:30 to 14:30)
  8hr_Driver_1 - 15-minute break (07:45 to 08:00)
  8hr_Driver_1 - 15-minute break (09:45 to 10:00)
  8hr_Driver_1 - 15-minute break (11:45 to 12:00)
  8hr_Driver_1 - 15-minute break (13:45 to 14:00)
  8hr_Driver_2 - 15-minute break (08:00 to 08:15)
  8hr_Driver_2 - 15-minute break (10:00 to 10:15)
  8hr_Driver_2 - 15-minute break (12:00 to 12:15)
  8hr_Driver_2 - 15-minute break (14:00 to 14:15)
  8hr_Driver_3 - 15-minute break (11:30 to 11:45)
  8hr_Driver_3 - 15-minute break (13:30 to 13:45)
  8hr_Driver_3 - 15-minute break (15:30 to 15:45)
  8hr_Driver_3 - 15-minute break (17:30 to 17:45)
  8hr_Driver_4 - 15-minute break (12:15 to 12:30)
  8hr_Driver_4 - 15-minute 

### 5. Adding supplementary drivers (at least two drivers working per night and two drivers working day shift on Saturdays and Sundays) -Total (16 Drivers) - 4 on Weekends, and 8 during Week days

In [15]:
def generate_timetable_with_all_shifts():
    # Initialize drivers for night and day shifts (12-hour)
    night_drivers = deque(["12hr_Driver_1", "12hr_Driver_2", "12hr_Driver_3", "12hr_Driver_4", "12hr_Driver_5", "12hr_Driver_6"])  # 3 Night-shift drivers
    day_drivers = deque(["12hr_Day_Driver_1", "12hr_Day_Driver_2", "12hr_Day_Driver_3", "12hr_Day_Driver_4", "12hr_Day_Driver_5", "12hr_Day_Driver_6"])  # 3 Day-shift drivers
    eight_hour_drivers = [
        "8hr_Driver_1", "8hr_Driver_2",  # 6:00-14:00
        "8hr_Driver_3", "8hr_Driver_4"   # 10:00-18:00
    ]
    
    schedule = []  # Store the timetable
    days = [
        "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
        "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
        "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
    ]
    
    # Rest pools for night shift and day shift drivers
    night_rest_pool = deque()
    day_rest_pool = deque()

    # Store the working drivers for each day (now includes shift times)
    day_drivers_schedule = {}

    for day in days:
        schedule.append(f"{day}:")
        day_drivers_schedule[day] = []  # Initialize the list for each day's working drivers

        # Reset lunch and break assignments at the start of each day
        lunch_drivers = []  # Reset lunch drivers for the day

        night_shift_drivers = []
        day_12_shift_drivers = []


        #Assign two drivers per night shift
       # Assign two drivers for the night shift
        for _ in range(2):
            # If no drivers are available, refill the deque from the rest pool
            if not night_drivers and night_rest_pool:
                while night_rest_pool:
                    rested_driver = night_rest_pool.popleft()["driver"]
                    night_drivers.append(rested_driver)

            # Safely assign a night driver
            if night_drivers:
                current_night_driver = night_drivers.popleft()
                night_shift_drivers.append(current_night_driver)
                schedule.append(f"  {current_night_driver} - night shift (18:00 to 6:00)")
                night_rest_pool.append({"driver": current_night_driver, "rest_days": []})
            else:
                schedule.append("  No night driver available")

        # 8-hour day shift assignment (6:00-18:00 and 10:00-18:00)
        if "Saturday" not in day and "Sunday" not in day:  # Only Monday to Friday
            for i in range(4):  # First four 8-hour drivers (6:00-18:00 and 10:00-18:00)
                if i < 2:
                    current_8hr_driver = eight_hour_drivers[i]
                    schedule.append(f"  {current_8hr_driver} - day shift (6:00 to 14:00)")
                    day_drivers_schedule[day].append((current_8hr_driver, "6:00 to 14:00"))
                else:
                    current_8hr_driver = eight_hour_drivers[i]
                    schedule.append(f"  {current_8hr_driver} - day shift (10:00 to 18:00)")
                    day_drivers_schedule[day].append((current_8hr_driver, "10:00 to 18:00"))

        # 12-hour day shift assignment (for Monday-Friday)
        if "Saturday" not in day and "Sunday" not in day:
            for _ in range(2):
                if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                    rested_day_driver = day_rest_pool.popleft()["driver"]
                    day_drivers.append(rested_day_driver)

                if day_drivers:
                    current_day_shift_driver = day_drivers.popleft()
                    schedule.append(f"  {current_day_shift_driver} - day shift (6:00 to 18:00)")
                    day_drivers_schedule[day].append((current_day_shift_driver, "6:00 to 18:00"))

                    day_rest_pool.append({"driver": current_day_shift_driver, "rest_days": []})

        # Day shift assignment for weekends (Saturdays and Sundays)
        if "Saturday" in day or "Sunday" in day:
            for _ in range(2):  # Two drivers on weekend days
                if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                    rested_day_driver = day_rest_pool.popleft()["driver"]
                    day_drivers.append(rested_day_driver)

                if day_drivers:
                    current_day_driver = day_drivers.popleft()
                    #day_12_shift_drivers.append()
                    schedule.append(f"  {current_day_driver} - day shift (6:00 to 18:00)")
                    day_drivers_schedule[day].append((current_day_driver, "6:00 to 18:00"))

                    day_rest_pool.append({"driver": current_day_driver, "rest_days": []})

        # Increment rest days for night-shift drivers
        for resting_driver in night_rest_pool:
            resting_driver["rest_days"].append(1)

        # Increment rest days for day-shift drivers
        for resting_driver in day_rest_pool:
            resting_driver["rest_days"].append(1)

        # Assign staggered lunch breaks
        lunch_drivers = assign_lunch_breaks(schedule, day_drivers_schedule[day])

        # Assign staggered 15-minute breaks for drivers without lunch
        assign_staggered_15_minute_breaks(schedule, lunch_drivers, day_drivers_schedule[day])

          # Assign staggered 15-minute breaks for night-shift drivers
        assign_staggered_15_minute_breaks_night(schedule, [(d, "18:00", "6:00") for d in night_shift_drivers])

    return schedule


def assign_lunch_breaks(schedule, working_drivers):
    """
    Assign lunch breaks based on the day's working drivers.
    """
    
    lunch_drivers_today = []

    # Assign lunch for the 12-hour day shift driver (6:00–18:00)
    twelve_hour_day_driver = [
        driver for driver, start_time in working_drivers if "12hr" in driver
    ]
    if twelve_hour_day_driver:
        # Assign lunch to the 12-hour shift worker from 13:00 to 14:00
        schedule.append(f"  {twelve_hour_day_driver[0]} - lunch break (13:00 to 14:00)")
        lunch_drivers_today.append(twelve_hour_day_driver[0])

        schedule.append(f"  {twelve_hour_day_driver[1]} - lunch break (14:00 to 15:00)")
        lunch_drivers_today.append(twelve_hour_day_driver[1])

    return lunch_drivers_today


def assign_staggered_15_minute_breaks(schedule, lunch_drivers, working_drivers):
    """
    Assign staggered 15-minute breaks to drivers not assigned lunch breaks.
    """
  
   # Initialize a list to store break assignments
    break_assignments = []
    
    # We will use a dictionary to track assigned break times to avoid conflicts
    assigned_breaks = {}

    # Filter out drivers who have lunch breaks
    eligible_drivers = [
        driver for driver, _ in working_drivers if driver not in lunch_drivers
    ]
    
    
    for driver, start_time in working_drivers:
        if driver not in eligible_drivers:
            continue

        # Parse shift start time
        driver_start_hour = int(start_time.split(":")[0])
        driver_end_hour = driver_start_hour + 8  # Assuming an 8-hour shift
        
        # List to store the assigned breaks for this driver
        breaks = []
        
        # Assign first break (2 hours after the start of the shift)
        first_break_hour = driver_start_hour + 2
        breaks.append((first_break_hour, 0))  # First break 2 hours after shift start
        
        # Now assign 3 more breaks, staggered at least 2 hours apart
        last_break = breaks[-1]
        while len(breaks) < 10:
            next_break_hour = last_break[0] + 2
            if next_break_hour >= driver_end_hour:  # Ensure break fits within the shift
                break
            breaks.append((next_break_hour, 0))
            last_break = breaks[-1]
        
        # Now, assign breaks to the schedule while ensuring no overlap
        for break_hour, break_minute in breaks:
            formatted_start = f"{break_hour:02}:{break_minute:02}"
            formatted_end = f"{break_hour:02}:{(break_minute + 15) % 60:02}"
            
            # Ensure the break time doesn't overlap with another driver
            while formatted_start in assigned_breaks:
                # If there's a conflict, move the break 15 minutes later
                break_minute += 15
                formatted_start = f"{break_hour:02}:{break_minute:02}"
                formatted_end = f"{break_hour:02}:{(break_minute + 15) % 60:02}"
            
            # Add the break to the schedule and track the assigned time
            break_assignments.append(f"  {driver} - 15-minute break ({formatted_start} to {formatted_end})")
            assigned_breaks[formatted_start] = driver
    
    # Append the break assignments to the schedule
    schedule.extend(break_assignments)



def assign_staggered_15_minute_breaks_night(schedule, night_drivers):
    """
    Assign staggered 15-minute breaks to night-shift drivers.
    Ensure no two drivers are on break at the same time.
    """
    break_assignments = []  # Store break assignments
    assigned_breaks = {}  # Track assigned break times to avoid conflicts

    
    for driver, shift_start, shift_end in night_drivers:
        # Parse the shift start and end hours
        start_hour = int(shift_start.split(":")[0])
        end_hour = int(shift_end.split(":")[0]) + 24  # Convert to 24-hour range for overnight shifts

        # List to store this driver's assigned breaks
        breaks = []

        # Assign breaks at 3-hour intervals, starting 2 hours into the shift
        for i in range(start_hour + 2, end_hour, 2):
            break_start = f"{i % 24:02}:00"
            break_end = f"{(i % 24):02}:15"

            # Ensure the break time doesn't overlap with another driver
            while break_start in assigned_breaks:
                # Move the break 15 minutes later if there's a conflict
                break_minute = int(break_start.split(":")[1]) + 15
                break_start = f"{i % 24:02}:{break_minute:02}"
                break_end = f"{i % 24:02}:{(break_minute + 15) % 60:02}"

            # Assign the break
            breaks.append(f"  {driver} - 15-minute break ({break_start} to {break_end})")
            assigned_breaks[break_start] = driver

        # Add this driver's breaks to the overall assignments
        break_assignments.extend(breaks)

    # Append the break assignments to the schedule
    schedule.extend(break_assignments)


# Generate the updated timetable
timetable = generate_timetable_with_all_shifts()

# Print the combined timetable in the requested format
for entry in timetable:
    print(entry)


Monday (Week 1):
  12hr_Driver_1 - night shift (18:00 to 6:00)
  12hr_Driver_2 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (6:00 to 14:00)
  8hr_Driver_2 - day shift (6:00 to 14:00)
  8hr_Driver_3 - day shift (10:00 to 18:00)
  8hr_Driver_4 - day shift (10:00 to 18:00)
  12hr_Day_Driver_1 - day shift (6:00 to 18:00)
  12hr_Day_Driver_2 - day shift (6:00 to 18:00)
  12hr_Day_Driver_1 - lunch break (13:00 to 14:00)
  12hr_Day_Driver_2 - lunch break (14:00 to 15:00)
  8hr_Driver_1 - 15-minute break (08:00 to 08:15)
  8hr_Driver_1 - 15-minute break (10:00 to 10:15)
  8hr_Driver_1 - 15-minute break (12:00 to 12:15)
  8hr_Driver_2 - 15-minute break (08:15 to 08:30)
  8hr_Driver_2 - 15-minute break (10:15 to 10:30)
  8hr_Driver_2 - 15-minute break (12:15 to 12:30)
  8hr_Driver_3 - 15-minute break (12:30 to 12:45)
  8hr_Driver_3 - 15-minute break (14:00 to 14:15)
  8hr_Driver_3 - 15-minute break (16:00 to 16:15)
  8hr_Driver_4 - 15-minute break (12:45 to 12:00)
  8hr_Driver_4 - 15

### 6. Putting results in tabular format

In [30]:
import pandas as pd

# Define table structures
days = [
    "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
    "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
    "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
]

# Prepare tables for output
drivers_schedule = {
    "Driver": [],
    **{day: [] for day in days}
}
lunch_breaks = {
    "Driver": [],
    **{day: [] for day in days}
}
fifteen_min_breaks = {
    "Driver": [],
    **{day: [] for day in days}
}

# Collect timetable data from the generated schedule
timetable_data = generate_timetable_with_all_shifts()

# Process timetable into structured data
current_driver = None

# Initialize variables to track the current day
current_day = None

for line in timetable_data:
    line = line.strip()  # Clean up leading/trailing spaces
    line_cleaned = line.rstrip(":")  # Remove trailing colons if present

    if not line:
        continue  # Skip empty lines

    # Detect a new day
    if line_cleaned in days:
        current_day = line_cleaned
        print(f"Current day set to: {current_day}")
        continue  # Move to the next line

    if current_day is None:
        print(f"Skipping line as no day is set: {line}")
        continue  # Skip lines if no day is set yet

    # Process shift details
    if "night shift" in line or "day shift" in line:
        shift_type = "Night Shift" if "night shift" in line else "Day Shift"
        driver = line.split(" - ")[0].strip()
        time_range = line.split("(")[-1].strip(")")

        # Initialize driver if not already in drivers_schedule
        if driver not in drivers_schedule["Driver"]:
            drivers_schedule["Driver"].append(driver)
            print(f"Added driver {driver} to drivers_schedule.")
            for day in days:
                if day not in drivers_schedule:
                    print(f"Initializing day {day} in drivers_schedule.")
                    drivers_schedule[day] = []
                drivers_schedule[day].append("")  # Initialize all days with empty strings

        # Update the current day for the driver
        driver_index = drivers_schedule["Driver"].index(driver)
        drivers_schedule[current_day][driver_index] = f"{shift_type}: {time_range}"
        print(f"Updated {current_day} for driver {driver}: {drivers_schedule[current_day][driver_index]}")

    # Process lunch breaks
    elif "lunch break" in line:
        driver = line.split(" - ")[0].strip()
        time_range = line.split("(")[-1].strip(")")

        # Initialize driver if not already in lunch_breaks
        if driver not in lunch_breaks["Driver"]:
            lunch_breaks["Driver"].append(driver)
            print(f"Added driver {driver} to lunch_breaks.")
            for day in days:
                if day not in lunch_breaks:
                    print(f"Initializing day {day} in lunch_breaks.")
                    lunch_breaks[day] = []
                lunch_breaks[day].append("")  # Initialize all days with empty strings

        # Update the current day for the driver
        driver_index = lunch_breaks["Driver"].index(driver)
        lunch_breaks[current_day][driver_index] = time_range
        print(f"Updated {current_day} for driver {driver}: {lunch_breaks[current_day][driver_index]}")

    # Process 15-minute breaks
    elif "15-minute break" in line:
        driver = line.split(" - ")[0].strip()
        time_range = line.split("(")[-1].strip(")")

        # Initialize driver if not already in fifteen_min_breaks
        if driver not in fifteen_min_breaks["Driver"]:
            fifteen_min_breaks["Driver"].append(driver)
            print(f"Added driver {driver} to fifteen_min_breaks.")
            for day in days:
                if day not in fifteen_min_breaks:
                    print(f"Initializing day {day} in fifteen_min_breaks.")
                    fifteen_min_breaks[day] = []
                fifteen_min_breaks[day].append("")  # Initialize all days with empty strings

        # Update the current day for the driver
        driver_index = fifteen_min_breaks["Driver"].index(driver)
        fifteen_min_breaks[current_day][driver_index] = time_range
        print(f"Updated {current_day} for driver {driver}: {fifteen_min_breaks[current_day][driver_index]}")


    """
    # Define dummy schedules for simplicity
    driver_schedule = {
        "Driver": ["Driver A", "Driver B", "Driver C"],
        "Monday (Week 1)": ["Day Shift (6:00 AM - 2:00 PM)", "", "Night Shift (10:00 PM - 6:00 AM)"],
        "Tuesday (Week 1)": ["Day Shift (6:00 AM - 2:00 PM)", "", ""],
        "Wednesday (Week 1)": ["", "Day Shift (6:00 AM - 2:00 PM)", ""],
    }

    lunch_break_schedule = {
        "Driver": ["Driver A", "Driver B", "Driver C"],
        "Monday (Week 1)": ["11:30 AM - 12:00 PM", "", "1:00 AM - 1:30 AM"],
        "Tuesday (Week 1)": ["11:30 AM - 12:00 PM", "", ""],
        "Wednesday (Week 1)": ["", "11:30 AM - 12:00 PM", ""],
    }

    fifteen_min_break_schedule = {
        "Driver": ["Driver A", "Driver B", "Driver C"],
        "Monday (Week 1)": ["9:00 AM - 9:15 AM", "", "2:30 AM - 2:45 AM"],
        "Tuesday (Week 1)": ["9:00 AM - 9:15 AM", "", ""],
        "Wednesday (Week 1)": ["", "9:00 AM - 9:15 AM", ""],
    }
    """

# Convert dictionaries to DataFrames
drivers_df = pd.DataFrame(drivers_schedule)
lunch_breaks_df = pd.DataFrame(lunch_breaks)
fifteen_min_breaks_df = pd.DataFrame(fifteen_min_breaks)

columns = ["Driver"] + days

# Restructure data to create the desired layout for shifts (Day & Night)
day_shift_rows = [["Day Shift"] + [""] * (len(columns) - 1)]    # Header for Day Shift
night_shift_rows = [["Night Shift"] + [""] * (len(columns) - 1)]  # Header for Night Shift

for _, row in drivers_df.iterrows():
    if any("Day Shift" in shift for shift in row[1:]):
        day_shift_rows.append([row["Driver"]] + list(row[1:]))
    if any("Night Shift" in shift for shift in row[1:]):
        night_shift_rows.append([row["Driver"]] + list(row[1:]))

# Combine into the final timetable structure for drivers
final_rows = day_shift_rows + night_shift_rows

# Create the final DataFrame for shifts
final_df = pd.DataFrame(final_rows, columns=columns)

# Filter and restructure the breaks data to include only relevant information
# For lunch breaks, include only non-empty values, skipping the "Driver" column
lunch_breaks_filtered = lunch_breaks_df[lunch_breaks_df.drop(columns=["Driver"]).notna().any(axis=1)]

# For 15-minute breaks, include only non-empty values, skipping the "Driver" column
fifteen_min_breaks_filtered = fifteen_min_breaks_df[fifteen_min_breaks_df.drop(columns=["Driver"]).notna().any(axis=1)]

# Convert filtered DataFrames to lists of rows for easy concatenation
lunch_breaks_rows = lunch_breaks_filtered.values.tolist()
fifteen_min_breaks_rows = fifteen_min_breaks_filtered.values.tolist()

# Combine headers and their respective rows
lunch_rows = [["Lunch Break"] + [""] * (len(columns) - 1)]  # Header for lunch breaks
min_break_rows = [["15 min breaks"] + [""] * (len(columns) - 1)]  # Header for 15 min breaks
final_breaks_rows = lunch_rows + lunch_breaks_rows + min_break_rows + fifteen_min_breaks_rows

# Create a DataFrame from the combined rows
breaks_final_df = pd.DataFrame(final_breaks_rows, columns=columns)


# Save to Excel
file_path = r"C:\Users\khaye\OneDrive\Desktop\Algorithms\5 semester\Driver_Schedule_Shifts.xlsx"
with pd.ExcelWriter(file_path) as writer:
    final_df.to_excel(writer, sheet_name="Driver Timetable", index=False)
    breaks_final_df.to_excel(writer, sheet_name="Breaks Schedule", index=False)

print("Drivers Schedule:", pd.DataFrame(drivers_schedule))
print("Lunch Breaks:", pd.DataFrame(lunch_breaks))
print("15-Min Breaks:", pd.DataFrame(fifteen_min_breaks))




Current day set to: Monday (Week 1)
Added driver 12hr_Driver_1 to drivers_schedule.
Updated Monday (Week 1) for driver 12hr_Driver_1: Night Shift: 18:00 to 6:00
Added driver 12hr_Driver_2 to drivers_schedule.
Updated Monday (Week 1) for driver 12hr_Driver_2: Night Shift: 18:00 to 6:00
Added driver 8hr_Driver_1 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_1: Day Shift: 6:00 to 14:00
Added driver 8hr_Driver_2 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_2: Day Shift: 6:00 to 14:00
Added driver 8hr_Driver_3 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_3: Day Shift: 10:00 to 18:00
Added driver 8hr_Driver_4 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_4: Day Shift: 10:00 to 18:00
Added driver 12hr_Day_Driver_1 to drivers_schedule.
Updated Monday (Week 1) for driver 12hr_Day_Driver_1: Day Shift: 6:00 to 18:00
Added driver 12hr_Day_Driver_2 to drivers_schedule.
Updated Monday (Week 1) for driver 12hr_Day_D

In [ ]:
import random
from collections import deque

# Constants
NUM_BUSES = 8
ROUTE_DURATION = 90  # 1.5 hours in minutes
POPULATION_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.1

# Initialize driver pools
drivers_12hr = ["Driver_12hr_" + str(i + 1) for i in range(10)]
drivers_8hr = ["Driver_8hr_" + str(i + 1) for i in range(10)]

# Helper functions
def calculate_route_duration(time_of_day):
    """Calculate route duration based on the time of day."""
    variability = random.randint(-10, 10)
    rush_hour_extra = 5 if time_of_day in ["7:00", "8:00", "9:00", "17:00", "18:00", "19:00"] else 0
    return ROUTE_DURATION + variability + rush_hour_extra

def generate_schedule():
    """Generate a random schedule for drivers."""
    schedule = []
    for bus in range(NUM_BUSES):
        bus_schedule = []
        current_time = 0  # Start of the day in minutes (0 = 00:00)
        while current_time < 1440:  # 1440 minutes = 24 hours
            driver = random.choice(drivers_12hr + drivers_8hr)
            if "12hr" in driver:
                shift_duration = 720  # 12 hours in minutes
                rest_days = 2  # Rest for 2 days after shift
            else:
                shift_duration = 480  # 8 hours in minutes
                rest_days = 0  # No mandatory rest days for 8-hour drivers
            
            start_time = f"{current_time // 60:02}:{current_time % 60:02}"
            end_time = f"{(current_time + shift_duration) // 60 % 24:02}:{(current_time + shift_duration) % 60:02}"
            bus_schedule.append((driver, start_time, end_time, rest_days))
            current_time += shift_duration + 15  # Include shift change
        schedule.append(bus_schedule)
    return schedule

def fitness(schedule):
    """Evaluate the fitness of a schedule."""
    total_drivers = len(set(driver for bus in schedule for driver, _, _, _ in bus))
    all_buses_covered = all(len(bus) > 0 for bus in schedule)
    return total_drivers if all_buses_covered else float("inf")

def crossover(schedule1, schedule2):
    """Perform crossover between two schedules."""
    crossover_point = random.randint(0, NUM_BUSES - 1)
    child1 = schedule1[:crossover_point] + schedule2[crossover_point:]
    child2 = schedule2[:crossover_point] + schedule1[crossover_point:]
    return child1, child2

def mutate(schedule):
    """Mutate a schedule."""
    for bus in schedule:
        if random.random() < MUTATION_RATE:
            random_index = random.randint(0, len(bus) - 1)
            random_driver = random.choice(drivers_12hr + drivers_8hr)
            bus[random_index] = (random_driver, *bus[random_index][1:])

def genetic_algorithm():
    """Optimize the schedule using a genetic algorithm."""
    # Initialize population
    population = [generate_schedule() for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        # Evaluate fitness
        population = sorted(population, key=fitness)
        if fitness(population[0]) == NUM_BUSES:
            break  # Found optimal solution
        
        # Selection (elitism)
        new_population = population[:5]  # Preserve top 5 schedules

        # Crossover
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.choices(population[:20], k=2)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])

        # Mutation
        for individual in new_population:
            mutate(individual)

        population = new_population

    # Return the best schedule
    return population[0]

# Run the genetic algorithm to find the optimal schedule
optimal_schedule = genetic_algorithm()
optimal_schedule


[[('Driver_8hr_9', '00:00', '12:00', 2),
  ('Driver_8hr_2', '12:15', '00:15', 2)],
 [('Driver_12hr_2', '00:00', '12:00', 2),
  ('Driver_12hr_2', '12:15', '20:15', 0),
  ('Driver_8hr_6', '20:30', '04:30', 0)],
 [('Driver_12hr_9', '00:00', '12:00', 2),
  ('Driver_12hr_8', '12:15', '00:15', 2)],
 [('Driver_8hr_3', '00:00', '12:00', 2),
  ('Driver_12hr_5', '12:15', '00:15', 2)],
 [('Driver_8hr_3', '00:00', '12:00', 2),
  ('Driver_8hr_10', '12:15', '00:15', 2)],
 [('Driver_8hr_10', '00:00', '08:00', 0),
  ('Driver_8hr_7', '08:15', '16:15', 0),
  ('Driver_8hr_4', '16:30', '00:30', 0)],
 [('Driver_8hr_7', '00:00', '12:00', 2),
  ('Driver_12hr_3', '12:15', '00:15', 2)],
 [('Driver_12hr_10', '00:00', '12:00', 2),
  ('Driver_12hr_10', '12:15', '00:15', 2)]]

In [24]:
from datetime import datetime, timedelta

# Constants
WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
WEEKENDS = ['Saturday', 'Sunday']
SHIFT_8_HOURS = timedelta(hours=8)
SHIFT_12_HOURS = timedelta(hours=12)

# Helper function to create time blocks for shifts
def create_time_blocks(start_time, shift_duration):
    start = datetime.strptime(start_time, "%H:%M")
    end = start + shift_duration
    return start, end

# Helper function to assign breaks based on the shift type
def assign_breaks(driver, start, end, break_type):
    """Assign breaks to the driver during their shift."""
    breaks = []
    if break_type == "15-min":
        # Stagger 15-min breaks every 1hr 45 mins (The route is 1hr 45mins long max)
        current = start + timedelta(minutes=105)

        # Check if the driver is a 12-hour driver
        if driver.shift_type == "12-hour":
            break_duration = timedelta(minutes=18)  # Set the break duration to 18 minutes for 12-hour drivers
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M")))
                current += timedelta(minutes=120)  # Move to the next break after 1 hour 45 minutes
        else:
            # For other drivers, use the usual break duration (e.g., 10 minutes)
            break_duration = timedelta(minutes=15)  # 10-minute breaks for other drivers
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M")))
                current += timedelta(minutes=120)  # Move to the next break after 1 hour 45 minutes

    elif break_type == "1-hour":
        # 1-hour break during a lunch window
        lunch_start = datetime.strptime("13:00", "%H:%M")
        lunch_end = datetime.strptime("15:00", "%H:%M")
        if start <= lunch_start < end:
            breaks.append((lunch_start.strftime("%H:%M"), (lunch_start + timedelta(hours=1)).strftime("%H:%M")))
        elif lunch_start <= start < lunch_end:
            breaks.append((start.strftime("%H:%M"), (start + timedelta(hours=1)).strftime("%H:%M")))
    return breaks

# Driver class
class Driver:
    def __init__(self, driver_id, shift_type):
        self.driver_id = driver_id
        self.shift_type = shift_type
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

# Bus class
class Bus:
    def __init__(self, bus_id, bus_index):
        self.bus_id = bus_id
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.drivers = []  # List of drivers assigned to this bus
        self.bus_index = bus_index  # Add this attribute

# Create buses with unique indices
buses = [Bus(f"Bus_{i+1}", i) for i in range(8)]


# Create drivers (8-hour and 12-hour)
driver_count = 0
drivers = []

# Rest day counter for 12-hour drivers
rest_day_counter = {}

# Reset schedules and rest-day counter
for bus in buses:
    bus.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}

for driver in drivers:
    driver.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
    driver.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

# Allocate 8-hour drivers
def allocate_8hr_drivers():
    global driver_count

    for bus_index, bus in enumerate(buses):
        # Assign the first 4 buses to the 06:00 start group
        if bus_index < 4:
            shift_start_time = "06:00"
        else:  # Assign the last 4 buses to the 09:00 start group
            shift_start_time = "09:00"

        driver_count += 1
        driver = Driver(f"Driver_{driver_count}", "8-hour")
        drivers.append(driver)

        for day in WEEKDAYS:  # 8-hour drivers only work on weekdays
            # Stagger start times for buses in each group
            staggered_start_time = (
                datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % 4) * 15)
            ).strftime("%H:%M")
            start, end = create_time_blocks(staggered_start_time, SHIFT_8_HOURS)
            breaks = assign_breaks(driver, start, end, "15-min")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))


# Allocate 12-hour drivers with a fixed set per bus
def allocate_12hr_drivers():
    global driver_count
    rest_day_counter = {}

    # Step 1: Assign a fixed set of 12-hour drivers for each bus
    for bus_index, bus in enumerate(buses):
        # First four buses get one pool of 12-hour drivers
        if bus_index < 4:  
            for i in range(3):  # Assign 3 drivers per bus in the first group
                driver_count += 1
                driver = Driver(f"Driver_{driver_count}", "12-hour")
                bus.drivers.append(driver)  # Attach driver to this bus
                drivers.append(driver)  # Add to global driver list
                rest_day_counter[driver.driver_id] = 0  # Initialize rest days
        else:  # Remaining four buses get another pool of 12-hour drivers
            for i in range(3):  # Assign 3 drivers per bus in the second group
                driver_count += 1
                driver = Driver(f"Driver_{driver_count}", "12-hour")
                bus.drivers.append(driver)  # Attach driver to this bus
                drivers.append(driver)  # Add to global driver list
                rest_day_counter[driver.driver_id] = 0  # Initialize rest days

    # Step 2: Assign shifts per bus, ensuring one driver works per day with staggered times
    for bus in buses:
        # Rotate through drivers for each bus based on rest-day counter
        driver_index = 0  # To rotate among the drivers assigned to the bus

        for day in WEEKDAYS + WEEKENDS:
            # Find the next available driver (with rest_day_counter == 0)
            while rest_day_counter[bus.drivers[driver_index].driver_id] > 0:
                rest_day_counter[bus.drivers[driver_index].driver_id] -= 1
                driver_index = (driver_index + 1) % len(bus.drivers)  # Move to the next driver

            # Get the selected driver
            driver = bus.drivers[driver_index]

            # Determine start time
            if day in ["Saturday", "Sunday"]:  # Weekend shifts start at 06:00 for the first driver
                staggered_start_time = (datetime.strptime("06:00", "%H:%M") + timedelta(minutes=(bus.bus_index % 4) * 18)).strftime("%H:%M")
            else:  # Weekday shifts start at 14:00
                staggered_start_time = (datetime.strptime("14:00", "%H:%M") + timedelta(minutes=(bus.bus_index % 4) * 15)).strftime("%H:%M")

            # Create time blocks for the shift
            start, end = create_time_blocks(staggered_start_time, SHIFT_12_HOURS)

            # Assign breaks based on shift type
            if day in WEEKDAYS + ["Saturday", "Sunday"]:  # Day shifts (including weekends) get lunch breaks
                if start.hour == 6 and start.minute == 0:  # Only the driver starting at 06:00 gets a 1-hour lunch break
                    breaks = assign_breaks(driver, start, end, "1-hour")
                else:  # For other cases
                    breaks = assign_breaks(driver, start, end, "15-min")
            else:  # Night shifts (including weekends) get 15-minute breaks
                breaks = assign_breaks(driver, start, end, "15-min")

            # Add breaks and schedule to the driver and bus
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            # Update rest_day_counter for the selected driver
            rest_day_counter[driver.driver_id] = 2  # Set to rest for 2 days

            # Move to the next driver for the following day
            driver_index = (driver_index + 1) % len(bus.drivers)




# Extend the driver pool to include weekend-only 12-hour drivers
def allocate_weekend_12hr_drivers():
    global driver_count

    # Weekend-only drivers for the night shift
    for bus_index, bus in enumerate(buses):
        # Add two drivers per bus: one for Saturday and one for Sunday
        for day in WEEKENDS:
            driver_count += 1
            driver = Driver(f"Driver_{driver_count}", f"12-hour (Weekend {day})")
            bus.drivers.append(driver)  # Add to the bus's driver list
            drivers.append(driver)  # Add to the global driver list

            # Assign shifts for the weekend driver
            start_time = (datetime.strptime("18:03", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")
            start, end = create_time_blocks(start_time, SHIFT_12_HOURS)

            # Assign 15-minute breaks
            breaks = assign_breaks(driver, start, end, "15-min")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))


# Assign drivers
allocate_8hr_drivers()
allocate_12hr_drivers()
allocate_weekend_12hr_drivers()

# Output the schedule in the requested format
print("Final Bus Schedule:")
for bus in buses:
    print(f"\n{bus.bus_id} Weekly Schedule:")
    for day in WEEKDAYS + WEEKENDS:
        if bus.schedule[day]:
            print(f"  {day}:")
            for shift in bus.schedule[day]:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                driver_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Driver: {driver_id}")

print("\nFinal Driver Schedule:")
for driver in drivers:
    print(f"\n{driver.driver_id} ({driver.shift_type}) Weekly Schedule:")
    for day in WEEKDAYS + WEEKENDS:
        if driver.schedule[day]:
            print(f"  {day}:")
            for shift in driver.schedule[day]:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                bus_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Bus: {bus_id}")
            if driver.breaks[day]:
                print("    Breaks:")
                for b in driver.breaks[day]:
                    print(f"      {b[0]} - {b[1]}")

print(f"\nTotal Drivers Used: {driver_count}")


Final Bus Schedule:

Bus_1 Weekly Schedule:
  Monday:
    Time: 06:00 - 14:00, Driver: Driver_1
    Time: 14:00 - 02:00, Driver: Driver_9
  Tuesday:
    Time: 06:00 - 14:00, Driver: Driver_1
    Time: 14:00 - 02:00, Driver: Driver_10
  Wednesday:
    Time: 06:00 - 14:00, Driver: Driver_1
    Time: 14:00 - 02:00, Driver: Driver_11
  Thursday:
    Time: 06:00 - 14:00, Driver: Driver_1
    Time: 14:00 - 02:00, Driver: Driver_9
  Friday:
    Time: 06:00 - 14:00, Driver: Driver_1
    Time: 14:00 - 02:00, Driver: Driver_10
  Saturday:
    Time: 06:00 - 18:00, Driver: Driver_11
    Time: 18:03 - 06:03, Driver: Driver_33
  Sunday:
    Time: 06:00 - 18:00, Driver: Driver_9
    Time: 18:03 - 06:03, Driver: Driver_34

Bus_2 Weekly Schedule:
  Monday:
    Time: 06:15 - 14:15, Driver: Driver_2
    Time: 14:15 - 02:15, Driver: Driver_12
  Tuesday:
    Time: 06:15 - 14:15, Driver: Driver_2
    Time: 14:15 - 02:15, Driver: Driver_13
  Wednesday:
    Time: 06:15 - 14:15, Driver: Driver_2
    Time: 14:1